In [1]:
import os
import torch
import pandas as pd
import pickle
from collections import defaultdict

from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

from DictaBERTEmbeddings import DictaBERTEmbeddings
from query_prep import *

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {DEVICE}')

DATA_PATH = './data/'
# Load data
pickle_file = os.path.join(DATA_PATH, 'prepd_data.pkl')
data = pd.read_pickle(pickle_file)

Using device: cuda


In [3]:
data.head(5)

,book_id,type,unit,headline,combined_info,lang,lemmatized_headline,lemmatized_text,document,lemmatized_document
0,241112,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49אין עמים ...",he,"חזון עתיד תוקן , חג ישראל , חזון נביא , תנופה ...","ד . בן - גוריון י""ח אייר תש""ט 17 . 5 . 49 עם ר...","headline: חזון עתיד מתוקן, חגי ישראל, חזון הנב...","headline: חזון עתיד תוקן , חג ישראל , חזון נבי..."
1,241170,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle EastBy David Ben-Gurion ...,en,"israel middle east חברות ישראל באו""ם גלות בר כ...",israel middle eastby david ben gurion prime mi...,"headline: israel and the middle east, חברות יש...","headline: israel middle east חברות ישראל באו""ם..."
2,241180,ארכיון בן גוריון - מסמך,נאומים ומאמרים,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-...",he,חוק שירות ביטחון,"ישיבה ע' ו , 5 . 9 . 1949 חוק שירות ביטחון תש""...","headline: חוק שירות הביטחון, text: ישיבה ע' ו,...","headline: חוק שירות ביטחון, text: ישיבה ע' ו ,..."
3,241225,ארכיון בן גוריון - מסמך,נאומים ומאמרים,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,en,ambassador morgenthau save palestine jewry wor...,ambassador morgenthau save palestine jewry wor...,headline: how ambassador Morgenthau saved Pale...,headline: ambassador morgenthau save palestine...
4,241459,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחבאני מודה, שהדברים של מ...",he,"[ ח""א ? ] הטה מורחב , צה""ל , מערכה סיני , מלחמ...","12 . 12 . 57 ח""א הטה מורחב הודה , דבר משה , דב...","headline: [ח""א?] במטה המורחב, צה""ל, מערכת סיני...","headline: [ ח""א ? ] הטה מורחב , צה""ל , מערכה ס..."


In [4]:
# Convert dataframe rows to LangChain Documents
docs = [
    Document(page_content=row['lemmatized_document'], metadata={"id": row['book_id']})
    for idx, row in data.iterrows()
]

In [5]:
def split_text_by_words(text, max_words, overlap_words):
    # Split the text into words
    words = text.split()

    chunks = []
    start_idx = 0
    while start_idx < len(words):
        end_idx = min(start_idx + max_words, len(words))
        chunk = words[start_idx:end_idx]

        # Join the words back into text
        chunk_text = ' '.join(chunk)
        chunks.append(chunk_text)

        # Update the starting position with overlap
        start_idx += max_words - overlap_words

    return chunks

In [6]:
chunks = []
for doc in docs:
    temp_chunks = split_text_by_words(
        doc.page_content, max_words=230, overlap_words=30
    )
    chunks.extend([Document(page_content=chunk, metadata=doc.metadata) for chunk in temp_chunks])

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dicta-il/dictabert")
j = 0
for i, doc in enumerate(chunks):
    tokens = tokenizer.encode(doc.page_content, truncation=False)
    if (len(tokens) > 512):
        j += 1
        print(f"Chunk {i}: {len(tokens)} tokens")
print(j)

Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors


Chunk 2467: 516 tokens
Chunk 3314: 513 tokens
Chunk 7447: 513 tokens
Chunk 7448: 687 tokens
4


In [8]:
# Check the number of chunks created
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 7912


In [9]:
# Initialize DictaBERT embeddings
embedding_model = DictaBERTEmbeddings(model_name="dicta-il/dictabert")

Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
EMB_PATH = os.path.join(DATA_PATH, 'embedding_dictaBERT.pkl')
with open(EMB_PATH, 'wb') as f:
    pickle.dump(embedding_model, f)

In [11]:
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE
)

In [12]:
vectorstore.save_local("./faiss_dicta_index")

In [13]:
# Load FAISS index
vectorstore = FAISS.load_local("./faiss_dicta_index", 
                               embedding_model,
                                allow_dangerous_deserialization=True
                               )